# Data Wrangling for Random Sampling of Players

In [132]:
cd Desktop/Capstone Project 1 Data Files

[Errno 2] No such file or directory: 'Desktop/Capstone Project 1 Data Files'
/Users/dillonquan/Desktop/Capstone Project 1 Data Files


In [133]:
import pandas as pd
df = pd.read_csv('Seasons_Stats.csv')
pd.options.display.max_columns = None

In [134]:
#cleaning up the dataset for sampling purposes

df = df.iloc[:,1:]
df = df[df.Year >= 2000]
df = df.drop(['blanl','blank2'], axis=1)
df['Year1'] = df.Year

In [135]:
#casting these columns to be of type int

l = ['Year','Age', 'G', 'GS']
for feature in l:
    df[feature] = df[feature].astype(dtype ='int')

In [136]:
#This nested for loop converts the total stats for the season to per game stats for readability.

to_per_game = [ 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA','FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL','BLK', 'TOV', 'PF', 'PTS']
for index,player in df.iterrows():
    for per_game in to_per_game:
        df.loc[index,per_game] = df.loc[index,per_game]/df.loc[index,'G']

In [137]:
#Read in the team_win_ratio csv file that was scraped in the other jupyter notebook to merge it with
#the Season Stats dataframe

df_team = pd.read_csv('team_win_ratio.csv')
df_team = df_team.iloc[:, 2:]
df_team1 = df_team[['Team','Year', 'W/L%']]
df_team1 = df_team1.rename(columns={'Team':'Tm'})
df = pd.merge(df, df_team1, how ='left', on=['Tm', 'Year'])

#setup to take out players who were traded

x = df.groupby(['Year','Player']).count()
df = df.set_index(['Year','Player'])
df = df[x.Tm == 1]

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [138]:
#This cell here creates a new column called MVP and each player will have either a 1 if they won MVP that season or 
#a 0 if they did not

df['MVP'] = 0
MVP = ["Shaquille O'Neal*", 'Allen Iverson*', 'Tim Duncan','Tim Duncan', 'Kevin Garnett', 'Steve Nash', 'Steve Nash', 'Dirk Nowitzki', 'Kobe Bryant', 'LeBron James','LeBron James','Derrick Rose', 'LeBron James','LeBron James','Kevin Durant','Stephen Curry','Stephen Curry','Russell Westbrook']
Season = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]
MVP_winner = list(zip(MVP,Season))
for mvp, year in MVP_winner:
    df.loc[(year,mvp),'MVP'] = 1

In [139]:
#function to calculate the marginal PER from the MVP of that season and the league's average that season.
#takes in 2 dataframe

def marginal_PER(player, season_data):
    avg_PER = np.mean(season_data.PER)
    marginal_PER = float(player.PER) - avg_PER
    return marginal_PER

In [140]:
#calculate the marginal PER for each player
df = df.reset_index()

for i in range(2000, 2018):
    x = df[df.Year == i]
    for index, player in x.iterrows():
        df.loc[index,'M_PER'] = marginal_PER(player, x)

The following cell will randomly sample 99 players from each year from the non_mvps dataframe. This randomly sampled dataframe will then be concatenated with the mvp dataframe to have a dataset that is ready for classification.

In [141]:
#seperate mvps and non mvps into two different dataframes

df = df.set_index(['Year', 'Player'])
df_mvps = df[df.MVP == 1]
df_non_mvps = df[df.MVP == 0]

In [142]:
#df1 contains the randomly sampled data that will be used to train our model with to classify MVPs and non MVPs

df1 = pd.DataFrame()
for year in range(2000,2018):
    random_sample = df_non_mvps.loc[(year,slice(None))].sample(99, replace = False, random_state = 4)
    df1 = pd.concat([df1,random_sample], axis = 0)
    
df1 = df1.reset_index()
df1 = df1.rename(columns={'Year1':'Year'})
df1= df1.set_index(['Player','Year'])
df_mvps = df_mvps.swaplevel()
df1 = pd.concat([df1, df_mvps], axis = 0)
df1 = df1.drop(['Year1'], axis = 1)

In [143]:
df1.head()

,,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,Age,BLK,BLK%,BPM,DBPM,DRB,DRB%,DWS,FG,FG%,FGA,FT,FT%,FTA,FTr,G,GS,MP,MVP,M_PER,OBPM,ORB,ORB%,OWS,PER,PF,PTS,Pos,STL,STL%,TOV,TOV%,TRB,TRB%,TS%,Tm,USG%,VORP,W/L%,WS,WS/48,eFG%
Player,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kevin Willis,2000.0,2.974684,0.415,7.164557,0.012658,0.333,0.037975,0.005,0.620253,4.8,37,0.607595,2.0,-4.4,-1.2,3.556962,19.4,1.6,2.987342,0.415,7.202532,1.658228,0.799,2.075949,0.288,79,1,21.253165,0,-0.346715,-3.1,2.544304,13.0,0.7,12.3,3.240506,7.645570,PF,0.455696,1.1,1.240506,13.3,6.101266,16.1,0.471,TOR,19.5,-1.0,0.549,2.3,0.066,0.416
Kenny Anderson,2000.0,4.256098,0.456,9.341463,1.036585,0.386,2.682927,0.223,5.121951,26.7,29,0.097561,0.2,1.2,-1.4,2.073171,7.9,1.9,5.292683,0.440,12.024390,2.390244,0.775,3.085366,0.257,82,82,31.621951,0,4.753285,2.6,0.670732,2.3,5.4,17.4,2.804878,14.012195,PG,1.695122,2.7,1.585366,10.6,2.743902,4.9,0.524,BOS,20.5,2.1,0.427,7.3,0.136,0.483
Randy Brown,2000.0,2.610169,0.359,7.271186,0.050847,0.500,0.101695,0.014,3.423729,22.6,31,0.254237,0.7,-5.7,-0.5,2.050847,8.9,1.2,2.661017,0.361,7.372881,1.050847,0.738,1.423729,0.193,59,55,27.542373,0,-5.346715,-5.2,0.389831,1.6,-2.0,7.3,2.033898,6.423729,PG,1.033898,2.0,1.779661,18.2,2.440678,5.2,0.402,CHI,16.2,-1.5,0.207,-0.8,-0.024,0.364
Allen Iverson*,2000.0,9.142857,0.435,21.028571,1.271429,0.341,3.728571,0.151,4.685714,23.0,24,0.071429,0.1,2.6,-0.8,2.800000,7.7,3.6,10.414286,0.421,24.757143,6.314286,0.713,8.857143,0.358,70,70,40.757143,0,7.353285,3.4,1.014286,2.7,3.3,20.0,2.314286,28.414286,SG,2.057143,2.6,3.285714,10.3,3.814286,5.2,0.496,PHI,34.4,3.3,0.598,6.9,0.116,0.446
Vince Carter,2000.0,8.451220,0.475,17.804878,1.158537,0.403,2.878049,0.139,3.926829,20.5,23,1.121951,2.1,4.6,-0.6,3.975610,12.1,2.7,9.609756,0.465,20.682927,5.317073,0.791,6.719512,0.325,82,82,38.121951,0,10.753285,5.2,1.829268,5.2,9.1,23.4,3.207317,25.695122,SF,1.341463,1.8,2.170732,8.4,5.804878,8.5,0.543,TOR,30.0,5.2,0.549,11.8,0.182,0.493
